In [1]:
import json
import hashlib
from datetime import datetime
from breeze_connect import BreezeConnect
import urllib
import pandas as pd
import numpy as np
import math
from trading_api.icici import icici
import datetime as dt

In [2]:
api_session='37289997'

In [5]:
icici1 = icici(api_session=api_session)
icici1.login()

In [85]:
stock = 'ITC'
from_date = '2024-03-19'
to_date = '2024-03-20'
interval = '1minute'
exchange = 'NSE'

df = icici1.get_histroy_data_stock(stock,from_date,to_date,interval=interval,exchange=exchange)


couldnt perform on datetime
Error could not convert string to float: '2024-03-19 09:08:00'
couldnt perform on stock_code
Error could not convert string to float: 'ITC'
open complete
close complete
high complete
low complete


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    398 non-null    object 
 1   stock_code  398 non-null    object 
 2   open        398 non-null    float64
 3   close       398 non-null    float64
 4   high        398 non-null    float64
 5   low         398 non-null    float64
dtypes: float64(4), object(2)
memory usage: 18.8+ KB


In [87]:
df['date_time'] = pd.to_datetime(df['datetime'])

In [88]:
df

,datetime,stock_code,open,close,high,low,date_time
0,2024-03-19 09:08:00,ITC,418.00,418.00,418.00,418.00,2024-03-19 09:08:00
1,2024-03-19 09:15:00,ITC,418.15,418.40,418.40,418.00,2024-03-19 09:15:00
2,2024-03-19 09:16:00,ITC,418.40,418.15,418.95,418.00,2024-03-19 09:16:00
3,2024-03-19 09:17:00,ITC,418.35,418.65,418.75,418.10,2024-03-19 09:17:00
4,2024-03-19 09:18:00,ITC,418.65,419.00,419.30,418.45,2024-03-19 09:18:00
...,...,...,...,...,...,...,...
393,2024-03-19 15:56:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:56:00
394,2024-03-19 15:57:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:57:00
395,2024-03-19 15:58:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:58:00
396,2024-03-19 15:59:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:59:00


In [89]:
df.drop(df.index[(df.date_time.dt.hour> 14) & (df.date_time.dt.minute >19)],inplace=True)

In [90]:
df['change'] = df['close']-df['open']

In [91]:
df['per_change'] = (df['change']/df['open'])*100

In [92]:
df['per_change'].mean()

-0.006618667089363048

In [94]:
df['change'].mean()

-0.02752043596730205

In [95]:
for index,row in df.iterrows():
    trade = pd.DataFrame()
    print(row['datetime'])

2024-03-19 09:08:00
2024-03-19 09:15:00
2024-03-19 09:16:00
2024-03-19 09:17:00
2024-03-19 09:18:00
2024-03-19 09:19:00
2024-03-19 09:20:00
2024-03-19 09:21:00
2024-03-19 09:22:00
2024-03-19 09:23:00
2024-03-19 09:24:00
2024-03-19 09:25:00
2024-03-19 09:26:00
2024-03-19 09:27:00
2024-03-19 09:28:00
2024-03-19 09:29:00
2024-03-19 09:30:00
2024-03-19 09:31:00
2024-03-19 09:32:00
2024-03-19 09:33:00
2024-03-19 09:34:00
2024-03-19 09:35:00
2024-03-19 09:36:00
2024-03-19 09:37:00
2024-03-19 09:38:00
2024-03-19 09:39:00
2024-03-19 09:40:00
2024-03-19 09:41:00
2024-03-19 09:42:00
2024-03-19 09:43:00
2024-03-19 09:44:00
2024-03-19 09:45:00
2024-03-19 09:46:00
2024-03-19 09:47:00
2024-03-19 09:48:00
2024-03-19 09:49:00
2024-03-19 09:50:00
2024-03-19 09:51:00
2024-03-19 09:52:00
2024-03-19 09:53:00
2024-03-19 09:54:00
2024-03-19 09:55:00
2024-03-19 09:56:00
2024-03-19 09:57:00
2024-03-19 09:58:00
2024-03-19 09:59:00
2024-03-19 10:00:00
2024-03-19 10:01:00
2024-03-19 10:02:00
2024-03-19 10:03:00


In [101]:
nd = pd.DataFrame(columns=df.columns)

In [103]:
nd = nd.append(df.sample)

AttributeError: 'DataFrame' object has no attribute 'append'